# Fintech-AutoGen Project: Step-by-Step Implementation Guide

Welcome! This notebook will guide you through building the Fintech-AutoGen project from scratch. You will learn how to set up your environment, scaffold the project, and understand the purpose behind each component. Code is provided as comments—uncomment as you progress. Each step includes detailed explanations to help you understand the "why" behind the actions.

## Step 1: Install `uv`, Create a Virtual Environment, and Initialize the Project

We use [`uv`](https://github.com/astral-sh/uv) for fast Python dependency management and environment creation. `uv` can create a virtual environment and initialize your project, automatically generating a `pyproject.toml` file for you. You can also use `uv init` to quickly scaffold a new Python project with a virtual environment and `pyproject.toml` in one step.

**Why?**
- Isolates your project dependencies
- Ensures reproducibility
- `pyproject.toml` is created and managed automatically
- `uv init` is the fastest way to start a new project

**Instructions:**
- Open a terminal in your project folder.
- You can use either of the following approaches:

**Option 1: Manual Steps**
- Run each command below step by step:

In [ ]:
# TODO: Uncomment and run ONE of the following in your terminal (not in the notebook)
#
# First, install uv (if not already installed)
# pip install uv
#
# Recommended: Use uv init to scaffold your project, create a virtual environment, and generate pyproject.toml interactively
# uv init
#
# If you want to do it manually (advanced):
# uv venv .venv
# .venv\Scripts\Activate.ps1
# uv pip install --upgrade pip

## Step 2: Project Structure Overview

Let’s understand the files and folders you’ll create. This modular structure keeps code organized and maintainable.

- `fintech_app/` — Main application code
- `database/` — Database schema and helpers
- `knowledge_base/` — ChromaDB integration
- `agents/` — Agent logic
- `teams/` — Team orchestration
- `tools/` — Tool adapters
- `ui/` — User interface (e.g., Streamlit)
- `pyproject.toml` — Project metadata and dependencies
- `README.md` — Project documentation

**Why?**
- Separation of concerns
- Easier to test, debug, and extend
- Modern Python projects use `pyproject.toml` for dependency management

In [ ]:
# TODO: Uncomment to create folders (run in terminal)
# mkdir fintech_app database knowledge_base agents teams tools ui

## Step 3: Create `__init__.py` Files

Each Python package (folder) needs an `__init__.py` file. This tells Python to treat the folder as a package.

**Why?**
- Enables imports from these folders
- Required for package structure

In [ ]:
# TODO: Uncomment to create empty __init__.py files in each folder (run in terminal)
# New-Item fintech_app\__init__.py -ItemType File
# New-Item database\__init__.py -ItemType File
# New-Item knowledge_base\__init__.py -ItemType File
# New-Item agents\__init__.py -ItemType File
# New-Item teams\__init__.py -ItemType File
# New-Item tools\__init__.py -ItemType File
# New-Item ui\__init__.py -ItemType File

## Step 4: Database Setup

I have provided fintech.db to you already.

See all the tables and dummy data populated in it

## Step 5:  Knowledge Base Integration



I have provided all the files to be ingested into chroma db under folder kb_texts.

Copy it into knowledge_base folder

We’ll use ChromaDB for storing and retrieving financial knowledge. 
Let’s create a file for this integration.



In [ ]:
# TODO: Create knowledge_base/kb_setup.py and copy the following code as comments. This code should match the original project exactly:

"""
Knowledge Base setup module
Handles creation and population of ChromaDB vector database
"""

from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
import os
from pathlib import Path
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader

# Define knowledge base directory path
KB_DIR = Path(__file__).parent / "chroma_db"
KB_TEXTS_DIR = Path(__file__).parent / "kb_texts"

def initialize_knowledge_base():
    """Initialize ChromaDB knowledge base with initial data using LangChain document loaders"""
    print(f"Setting up ChromaDB knowledge base at {KB_DIR}...")
    os.makedirs(KB_DIR, exist_ok=True)
    os.makedirs(KB_TEXTS_DIR, exist_ok=True)

    # Load documents from text files, adding domain metadata
    docs = []
    for txt_file in KB_TEXTS_DIR.glob("*.txt"):
        domain = txt_file.stem  # e.g., 'tech_sector' from 'tech_sector.txt'
        loader = TextLoader(str(txt_file))
        loaded_docs = loader.load()
        # Add domain metadata to each document
        for doc in loaded_docs:
            if not hasattr(doc, 'metadata') or doc.metadata is None:
                doc.metadata = {}
            doc.metadata['domain'] = domain
        docs.extend(loaded_docs)
    # Create vectorstore using LangChain Chroma
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        docs,
        embedding=embeddings,
        persist_directory=str(KB_DIR),
        collection_name="fintech_knowledge"
    )
    #vectorstore.persist()
    print("Knowledge base setup complete!")



# For testing
if __name__ == "__main__":
    initialize_knowledge_base()

## Step 6: Scaffold Agent and Team Files

Agents encapsulate business logic. Teams orchestrate agent collaboration. Let’s create starter files for each.

**Why?**
- Modularizes logic for maintainability
- Supports parallel and sequential workflows

In [ ]:
# TODO: Create agent and team files as shown below 
# agents/portfolio_manager.py
# agents/market_analyst.py
# agents/financial_educator.py
# agents/risk_assessment.py
# agents/research_agent.py
# teams/financial_education_team.py
# teams/portfolio_optimization_team.py
# teams/market_research_team.py
# teams/orchestrator.py


# """

## Step 7: Scaffold Tool Adapters

Tool adapters connect your app to external APIs and services (e.g., ChromaDB, Yahoo Finance).

**Why?**
- Decouples business logic from external dependencies
- Makes testing and extension easier

In [ ]:
# TODO: Create tools/chromadb_adapter.py and tools/finance_api_adapter.py (copy as comments)
# tools/chromadb_adapter.py
# """
# # Adapter for ChromaDB operations
# # TODO: Implement functions for document retrieval
# """

# tools/finance_api_adapter.py
# """
# # Adapter for financial data APIs (e.g., Yahoo Finance)
# # TODO: Implement functions for fetching stock data
# """

## Step 8: Next Steps and Best Practices

You’ve scaffolded the project! Next, you’ll:
- Implement each module step by step
- Write tests for your code
- Build the UI and connect it to your backend

**Best Practices:**
- Commit early and often
- Write clear docstrings and comments
- Test each component as you go

---

**Congratulations!** You now have a solid foundation for your Fintech-AutoGen project. Continue to the next notebook or section for implementation details.